In [42]:
# use natural language toolkit
import nltk
from nltk.stem.lancaster import LancasterStemmer
import os
import json
import datetime
stemmer = LancasterStemmer()

In [43]:
# 3 classes of training data
training_data = []
training_data.append({"class":'Advantages',"sentence":'Because tasks can be completed within the browser without communicating with the server, JavaScript can create a smooth “desktop-like” experience for the end user'})
training_data.append({"class":'Advantages',"sentence":'From drag-and-drop blocks to stylized sliders, there are numerous ways that JavaScript can be used to enhance a website’s UI/UX.'})
training_data.append({"class":'Advantages',"sentence":'TObjects can inherit from other objects, which makes JavaScript so simple, powerful, and great for dynamic applications.'})
training_data.append({"class":'Advantages',"sentence":'js can be executed on client-side also'})
training_data.append({"class":'Advantages',"sentence":'javascript is fast.'})
training_data.append({"class":'Advantages',"sentence":'javascript` has rich interface'})
training_data.append({"class":'Advantages',"sentence":'js has many libraries'})
training_data.append({"class":'Advantages',"sentence":'JavaScript small bit of code you can test easily on Console Panel.'})
training_data.append({"class":'Advantages',"sentence":'JavaScript syntaxs are easy and flexible for the developers.'})
training_data.append({"class":'Advantages',"sentence":'The biggest advantages to a JavaScript having a ability to support all modern browser and produce the same result.'})
training_data.append({"class":'Advantages',"sentence":'JS has rich interface'})
training_data.append({"class":'Advantages',"sentence":'javascript executes on both client-side & server side'})
training_data.append({"class":'Advantages',"sentence":'javascript has the ability to inherit parent class behaviour to child class'})
training_data.append({"class":'Disadvantages',"sentence":'problem of clientside security'})
training_data.append({"class":'Disadvantages',"sentence":'javascript is having secuirty related issues'})
training_data.append({"class":'Disadvantages',"sentence": 'JavaScripts main weakness is security. Look for answers on how it can be exploited. A secondary weakness is JavaScripts ubiquity and versatility—it can be a double-edged sword in that theres a lot of room for programming quirks that can lead to inconsistent performance across different platforms.'})
training_data.append({"class":'Disadvantages',"sentence":'main weekness of the js is security'})
training_data.append({"class":'Disadvantages',"sentence":'security,ubiquity,versatility,clientside security'})
training_data.append({"class":'Disadvantages',"sentence":' Because the code executes on the users’ computer, in some cases it can be exploited for malicious purposes. This is one reason some people choose to disable Javascript'})
training_data.append({"class":'Disadvantages',"sentence":'JavaScript is sometimes interpreted differently by different browsers. Whereas server-side scripts will always produce the same output, client-side scripts can be a little unpredictable. Don’t be overly concerned by this though - as long as you test your script in all the major browsers you should be safe. Also, there are services out there that will allow you to test your code automatically on check in of an update to make sure all browsers support your code.'})
training_data.append({"class":'Differences',"sentence":' A constructor function instantiates an instance via the “new” keyword. This new instance inherits properties from a parent class.'})
training_data.append({"class":'Differences',"sentence": 'ClassInheritance: instances inherit from classes (like a blueprint — a description of the class), and create sub-class relationships prototypal Instances may be composed from many different objects, allowing for easy selective inheritance.'})
training_data.append({"class":'Differences',"sentence":'Class Inheritance: instances inherit from classes (like a blueprint — a description of the class), and create sub-class relationships: hierarchical class taxonomies. Instances are typically instantiated via constructor functions with the new keyword. Class inheritance may or may not use the class keyword from ES6.    Prototypal Inheritance: instances inherit directly from other objects. Instances are typically instantiated via factory functions or Object.create(). Instances may be composed from many different objects, allowing for easy selective inheritance.'})
training_data.append({"class":'Differences',"sentence": 'Classical Inheritance: A constructor function instantiates an instance via the “new” keyword. This new instance inherits properties from a parent class.  Prototypal Inheritance: An instance is created by cloning an existing object that serves as a prototype. This instance—often instantiated using a factory function or “Object.create()”—can benefit from selective inheritance from many different objects.'})
#training_data.append({"class":'Differences',"sentence":' A constructor function instantiates an instance via the “new” keyword. This new instance inherits properties from a parent class.'})
#training_data.append({"class":'Differences',"sentence":' An instance is created by cloning an existing object that serves as a prototype. This instance—often instantiated using a factory function or “Object.create()”—can benefit from selective inheritance from many different objects'})
print ("%s sentences in training data" % len(training_data))

24 sentences in training data


In [44]:
# capture unique stemmed words in the training corpus
class_words={}
corpus_words = {}
classes = list(set([a['class'] for a in training_data]))
for c in classes:
    class_words[c] = []
    
for data in training_data:
    # tokenize each sentence into words
    for word in nltk.word_tokenize(data['sentence']):
        # ignore a few things
        if word not in ["?", "'s"]:
            # stem and lowercase each word
            stemmed_word = stemmer.stem(word.lower())
            if stemmed_word not in corpus_words:
                corpus_words[stemmed_word] = 1
            else:
                corpus_words[stemmed_word] += 1
                
            class_words[data['class']].extend([stemmed_word])

# we now have each word and the number of occurances of the word in our training corpus (the word's commonality)
print ("Corpus words and counts: %s" % corpus_words)
# also we have all words in each class
print ("Class words: %s" % class_words)

Corpus words and counts: {'becaus': 2, 'task': 1, 'can': 11, 'be': 11, 'complet': 1, 'within': 1, 'the': 18, 'brows': 5, 'without': 1, 'commun': 1, 'with': 2, 'serv': 4, ',': 15, 'javascrib': 15, 'cre': 4, 'a': 18, 'smoo': 1, '“': 4, 'desktop-like': 1, '”': 4, 'expery': 1, 'for': 8, 'end': 1, 'us': 5, 'from': 12, 'drag-and-drop': 1, 'block': 1, 'to': 10, 'styl': 1, 'slid': 1, 'ther': 4, 'ar': 5, 'num': 1, 'way': 1, 'that': 5, 'enh': 1, 'websit': 1, '’': 3, 's': 1, 'ui/ux': 1, '.': 27, 'tobject': 1, 'inherit': 15, 'oth': 2, 'object': 6, 'which': 1, 'mak': 2, 'so': 1, 'simpl': 1, 'pow': 1, 'and': 6, 'gre': 1, 'dynam': 1, 'apply': 1, 'js': 4, 'execut': 3, 'on': 7, 'client-side': 3, 'also': 2, 'is': 8, 'fast': 1, '`': 1, 'has': 4, 'rich': 2, 'interfac': 2, 'many': 4, 'libr': 1, 'smal': 1, 'bit': 1, 'of': 7, 'cod': 4, 'you': 4, 'test': 3, 'easy': 4, 'consol': 1, 'panel': 1, 'syntax': 1, 'flex': 1, 'develop': 1, 'biggest': 1, 'adv': 1, 'hav': 2, 'abl': 2, 'support': 2, 'al': 3, 'modern': 1, 

In [45]:
# we can now calculate the Naive Bayes score for a new sentence
#sentence = "javascript is server-side programming language"
sentence = "From drag-and-drop blocks to stylized sliders, there are numerous ways that JavaScript can be used to enhance a website’s UI/UX"

# calculate a score for a given class
def calculate_class_score(sentence, class_name):
    score = 0
    for word in nltk.word_tokenize(sentence):
        if word in class_words[class_name]:
            score += 1
    return score

In [48]:
# now we can find the class with the highest score
for c in class_words.keys():
    print ("Class: %s  Score: %s" % (c, calculate_class_score(sentence, c)))

Class: Differences  Score: 4
Class: Advantages  Score: 10
Class: Disadvantages  Score: 8


In [49]:
# calculate a score for a given class taking into account word commonality
def calculate_class_score_commonality(sentence, class_name):
    score = 0
    for word in nltk.word_tokenize(sentence):
        if word in class_words[class_name]:
            score += (1 / corpus_words[word])
    return score

In [50]:
# now we can find the class with the highest score
for c in class_words.keys():
    print ("Class: %s  Score: %s" % (c, calculate_class_score_commonality(sentence, c)))

Class: Differences  Score: 0.4131313131313131
Class: Advantages  Score: 3.0373737373737373
Class: Disadvantages  Score: 1.0373737373737375


In [51]:
def find_class(sentence):
    high_class = None
    high_score = 0
    for c in class_words.keys():
        score = calculate_class_score_commonality(sentence, c)
        if score > high_score:
            high_class = c
            high_score = score
    return high_class, high_score

In [56]:
find_class("The biggest advantages to a JavaScript having a ability to support all modern browser and produce the same result.JavaScript small bit of code you can test easily on Console Panel.")

('Advantages', 5.030327080327081)